In [119]:
#(Requires D-wave's Ocean SDK)
import neal
import dimod
import numpy as np
from dimod.core.polysampler import ComposedPolySampler, PolySampler
from dimod.higherorder.polynomial import BinaryPolynomial
from dimod.higherorder.utils import make_quadratic, poly_energies
from dimod.sampleset import SampleSet
from collections import defaultdict
import collections

sampler = dimod.HigherOrderComposite(neal.SimulatedAnnealingSampler())

#The Ising cost function in h J of the allZ Hamiltonian from S4 (of the qwc fragment) 
h = {0:0.15542669077992818, 1: 0.1062290449085607, 2: 0.16326768673564326 }
J = {(0,1):0.15660062488237944, (0,2): 0.1062290449085607, (0,3): -0.04919764587136754,
    (1,2):0.15542669077992818, (2,3):0.04919764587136754,
    (0,1,3):0.04919764587136754, (1,2,3):-0.04919764587136754}

offset = -0.3276081896748086 #for the hamiltonian term that didn't have any operators associated with it

sampleset = sampler.sample_hising(h, J, discard_unsatisfied=True)
sample_energy = sampleset.record[0][1]
total_energy = sample_energy  + offset
print("The total energy is: ",total_energy)


The total energy is:  -0.7458717930355663


In [49]:
#Lets try that on an exact solver
sampler = dimod.HigherOrderComposite(dimod.ExactSolver())
sampleset = sampler.sample_hising(h, J, discard_unsatisfied=True)
print("Just add the offset value")
print(sampleset)


Just add the offset value
    0  1  2  3  energy num_oc. penalt.
6  +1 -1 -1 +1 -0.418264       1    True
25 +1 -1 -1 +1 -0.418264       1    True
1  -1 +1 -1 -1 -0.418264       1    True
9  -1 +1 -1 +1 -0.418264       1    True
22 -1 +1 -1 +1 -0.418264       1    True
30 -1 +1 -1 -1 -0.418264       1    True
12 -1 -1 +1 -1 -0.400234       1    True
19 -1 -1 +1 -1 -0.400234       1    True
14 +1 -1 -1 -1 -0.024682       1    True
17 +1 -1 -1 -1 -0.024682       1    True
8  +1 +1 -1 +1 -0.006667       1    True
23 +1 +1 -1 +1 -0.006667       1    True
15 -1 -1 -1 -1 -0.006667       1    True
16 -1 -1 -1 -1 -0.006667       1    True
7  -1 -1 -1 +1 -0.006667       1    True
24 -1 -1 -1 +1 -0.006667       1    True
0  +1 +1 -1 -1 -0.006667       1    True
31 +1 +1 -1 -1 -0.006667       1    True
4  -1 -1 +1 +1 -0.006653       1    True
27 -1 -1 +1 +1 -0.006653       1    True
2  -1 +1 +1 -1 0.006667       1    True
29 -1 +1 +1 -1 0.006667       1    True
5  +1 -1 +1 +1 0.006667       1    

In [178]:
#Now take a 12 variable random problem to see the effects of beta range
np.random.seed(43)
hrand = {}
Jrand = {}
init_st = {}
maxvar = 8
for i in range(0,maxvar):
    init_st[i] = -1
"""for i in range(0,maxvar):
    if np.random.random() > .5:
        init_st[i] =  1
    else:
        init_st[i] = -1"""
for i in range(0,maxvar):
    hrand[i] =  (1 + 1) * np.random.random() - 1
for i in range(0,maxvar):
    for j in range(i+1,maxvar):
        if np.random.random() > .95:
            Jrand[(i,j)] = (1 + 1) * np.random.random() - 1
        for k in range(j+1,maxvar):
            if np.random.random() > .95:
                Jrand[(i,j,k)] = (1 + 1) * np.random.random() - 1

#print(init_st)

#print(Jrand)

sampler_rand = dimod.HigherOrderComposite(neal.SimulatedAnnealingSampler())

sampleset_rand = sampler_rand.sample_hising(hrand,Jrand,discard_unsatisfied=False, num_reads=4,seed=4)
print("#####")
print("Default beta_range")
print(sampleset_rand)
#sampleset = sampler.sample_hising(hrand, Jrand, discard_unsatisfied=True, num_reads=4,initial_state={0:-1,1:-1,2:-1,3:-1},beta_range=[4.2, 4.2])
print("#####")
print("Beta_range 54 to 55, more localized search")
sampleset_rand = sampler_rand.sample_hising(hrand,Jrand,discard_unsatisfied=False, num_reads=4,beta_range=[54, 55],seed=4)
print(sampleset_rand)
#let's try the same with an init state
print(init_st)
sampleset_rand = sampler_rand.sample_hising(hrand,Jrand,discard_unsatisfied=False, num_reads=4,beta_range=[54, 55],initial_state=init_st,seed=4)
print("#####")
print("Beta range 54 to 55 and supp initial state")
print(sampleset_rand)

#####
Default beta_range
   0  1  2  3  4  5  6  7    energy num_oc. penalt.
0 +1 +1 +1 +1 +1 -1 -1 +1 -6.397377       1       1
1 +1 +1 +1 +1 +1 -1 -1 +1 -6.397377       1       1
2 +1 +1 +1 +1 +1 -1 -1 +1 -6.397377       1       1
3 +1 +1 +1 +1 +1 -1 -1 +1 -6.397377       1       1
['SPIN', 4 rows, 4 samples, 8 variables]
#####
Beta_range 54 to 55, more localized search
   0  1  2  3  4  5  6  7    energy num_oc. penalt.
0 +1 -1 +1 +1 +1 -1 +1 -1 -4.347092       1       0
2 +1 +1 +1 +1 -1 -1 -1 -1 -4.018964       1       0
1 +1 +1 +1 +1 +1 -1 -1 -1 -3.967919       1       0
3 +1 +1 +1 -1 +1 +1 +1 -1 -2.395826       1       1
['SPIN', 4 rows, 4 samples, 8 variables]
{0: -1, 1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1, 7: -1}
#####
Beta range 54 to 55 and supp initial state
   0  1  2  3  4  5  6  7    energy num_oc. penalt.
0 +1 -1 +1 +1 +1 -1 +1 -1 -4.347092       1       0
2 +1 +1 +1 +1 -1 -1 -1 -1 -4.018964       1       0
1 +1 +1 +1 +1 +1 -1 -1 -1 -3.967919       1       0
3 +1 +1 +1

In [179]:
#No luck with the initial state
#I guess we have to reduce down to binary polynomial

#This code is directly taken from OCEAN SDK, Credit goes to them
def expand_initial_state(bqm, initial_state):
    """Determine the values for the initial state for a binary quadratic model
    generated from a higher order polynomial.

    Args:
        bqm (:obj:`.BinaryQuadraticModel`): a bqm object that contains
            its reduction info.

        initial_state (dict):
            An initial state for the higher order polynomial that generated the
            binary quadratic model.

    Returns:
        dict: A fully specified initial state.

    """
    # Developer note: this function relies heavily on assumptions about the
    # existance and structure of bqm.info['reduction']. We should consider
    # changing the way that the reduction information is passed.
    if not bqm.info['reduction']:
        return initial_state  # saves making a copy

    initial_state = dict(initial_state)  # so we can edit it in-place

    for (u, v), changes in bqm.info['reduction'].items():

        uv = changes['product']
        initial_state[uv] = initial_state[u] * initial_state[v]

        if 'auxiliary' in changes:
            # need to figure out the minimization from the initial_state
            aux = changes['auxiliary']

            en = (initial_state[u] * bqm.adj[aux].get(u, 0) +
                  initial_state[v] * bqm.adj[aux].get(v, 0) +
                  initial_state[uv] * bqm.adj[aux].get(uv, 0))

            initial_state[aux] = min(bqm.vartype.value, key=lambda val: en*val)

    return initial_state

polyrand = {}
for key in hrand:
    polyrand[(key,)] = hrand[key]
for key in Jrand:
    polyrand[key] = Jrand[key]


bqm = make_quadratic(polyrand, 1.0, vartype=dimod.SPIN)
noveau_init_st = expand_initial_state(bqm, init_st)

#These dictionaries will hold the same bqm but with changed variable names
bqm_lin = {}
bqm_quad = {}
int2str_dict = {}
var_name_ctr = maxvar #instead of string names, we'll assign these
for key in bqm.linear:
    
    if isinstance(key, int):
        bqm_lin[key] = bqm.linear[key]
    else:
        bqm_lin[var_name_ctr] = bqm.linear[key]
        int2str_dict[key] = var_name_ctr
        var_name_ctr += 1

key0 = None
key1 = None
for key in bqm.quadratic:
    if isinstance(key[0], int):
        key0 = key[0]
    else:
        key0 = int2str_dict[key[0]]
    
    if isinstance(key[1], int):
        key1 = key[1]
    else:
        key1 = int2str_dict[key[1]]
    
    bqm_quad[(key0,key1)] = bqm.quadratic[key]

    
        
#print(var_name_ctr)
#print(bqm.linear)
#print("------")
#print(bqm_lin)
#print("------")
#print(bqm.quadratic)
#print("------")
#print(bqm_quad)
#print("------")
#print(int2str_dict)
#print(len(noveau_init_st))
#print(noveau_init_st)
#od = collections.OrderedDict(sorted(noveau_init_st.items()))
noveau_l = []
noveau_var_names = []
for key in noveau_init_st: #lets 'read' a sample out of the dict the same way it presents its variables
    noveau_l.append(noveau_init_st[key]) 
    noveau_var_names.append(key)

#print(noveau_init_st)
#All this to make a tuple structure called sample-like that will accept it
#noveau_sample_like = (np.array([noveau_l],dtype='int8'),noveau_var_names)
#print(noveau_sample_like)

In [188]:
#Okay, let's try that once more
newsampler = neal.SimulatedAnnealingSampler()

#default settings
response = newsampler.sample_ising(bqm.linear,bqm.quadratic,num_reads=4,seed=4)
print("########")
print("For default settings")
for i in range(0,len(response.record)):
    answer_sample = response.record[i][0][0:maxvar]
    print(answer_sample)
    print(poly_energies(answer_sample,polyrand))

#limited beta range
response = newsampler.sample_ising(bqm.linear,bqm.quadratic,num_reads=4,seed=4,beta_range=[54,55])
print("########")
print("For limited beta range")
for i in range(0,len(response.record)):
    answer_sample = response.record[i][0][0:maxvar]
    print(answer_sample)
    print(poly_energies(answer_sample,polyrand))

#limited beta range + init_state
response = newsampler.sample_ising(bqm_lin,bqm_quad,num_reads=4,seed=4,beta_range=[54,55],initial_states = dimod.as_samples(noveau_l))
print("########")
print("For limited beta range + init state")
for i in range(0,len(response.record)):
    answer_sample = response.record[i][0][0:maxvar]
    print(answer_sample)
    print(poly_energies(answer_sample,polyrand))

########
For default settings
[ 1  1  1  1  1 -1 -1  1]
[-6.39737679]
[ 1  1  1  1  1 -1 -1  1]
[-6.39737679]
[ 1  1  1  1  1 -1 -1  1]
[-6.39737679]
[ 1  1  1  1  1 -1 -1  1]
[-6.39737679]
########
For limited beta range
[ 1 -1  1  1  1 -1  1 -1]
[-4.34709184]
[ 1  1  1  1  1 -1 -1 -1]
[-3.96791932]
[ 1  1  1  1 -1 -1 -1 -1]
[-4.01896427]
[ 1  1  1 -1  1  1  1 -1]
[-2.39582633]
########
For limited beta range + init state
[ 1  1  1  1 -1 -1 -1 -1]
[-4.01896427]
[ 1 -1  1  1  1 -1  1 -1]
[-4.34709184]
[ 1  1  1  1 -1 -1 -1 -1]
[-4.01896427]
[ 1  1  1  1 -1 -1 -1 -1]
[-4.01896427]


In [185]:
#Taken from the neal package
def default_ising_beta_range(h, J):
    """Determine the starting and ending beta from h J
    Args:
        h (dict)
        J (dict)
    Assume each variable in J is also in h.
    We use the minimum bias to give a lower bound on the minimum energy gap, such at the
    final sweeps we are highly likely to settle into the current valley.
    """
    # Get nonzero, absolute biases
    abs_h = [abs(hh) for hh in h.values() if hh != 0]
    abs_J = [abs(jj) for jj in J.values() if jj != 0]
    abs_biases = abs_h + abs_J

    if not abs_biases:
        return [0.1, 1.0]

    # Rough approximation of min change in energy when flipping a qubit
    min_delta_energy = min(abs_biases)

    # Combine absolute biases by variable
    abs_bias_dict = defaultdict(int, {k: abs(v) for k, v in h.items()})
    for (k1, k2), v in J.items():
        abs_bias_dict[k1] += abs(v)
        abs_bias_dict[k2] += abs(v)

    # Find max change in energy when flipping a single qubit
    max_delta_energy = max(abs_bias_dict.values())

    # Selecting betas based on probability of flipping a qubit
    # Hot temp: We want to scale hot_beta so that for the most unlikely qubit flip, we get at least
    # 50% chance of flipping.(This means all other qubits will have > 50% chance of flipping
    # initially.) Most unlikely flip is when we go from a very low energy state to a high energy
    # state, thus we calculate hot_beta based on max_delta_energy.
    #   0.50 = exp(-hot_beta * max_delta_energy)
    #
    # Cold temp: Towards the end of the annealing schedule, we want to minimize the chance of
    # flipping. Don't want to be stuck between small energy tweaks. Hence, set cold_beta so that
    # at minimum energy change, the chance of flipping is set to 1%.
    #   0.01 = exp(-cold_beta * min_delta_energy)
    hot_beta = np.log(2) / max_delta_energy
    cold_beta = np.log(100) / min_delta_energy

    return [hot_beta, cold_beta]



In [186]:
default_ising_beta_range(bqm_lin, bqm_quad)

[0.10637263212208087, 55.93929396069965]

In [202]:
#The Ising cost function in h J of the allZ Hamiltonian from S4 (of the qwc fragment) 
h = {0:0.15542669077992818+0.13716572937099494, 1: 0.1062290449085607+0.13716572937099494, 2: 0.16326768673564326-0.1303629205710914,3:-0.13036292057109133}
J = {(0,1):0.15660062488237944, (0,2): 0.1062290449085607, (0,3): -0.04919764587136754,
    (1,2):0.15542669077992818, (2,3):0.04919764587136754,
    (0,1,3):0.04919764587136754, (1,2,3):-0.04919764587136754}
offset = -0.3276081896748086

sampler = dimod.HigherOrderComposite(dimod.ExactSolver())
sampleset = sampler.sample_hising(h, J, discard_unsatisfied=True)
print("Just add the offset value")
#print(sampleset)
print(sampleset.first.energy)
sampleset.first.energy + offset

Just add the offset value
-0.6745652019864152


-1.0021733916612239

In [203]:
h = {0:0.13716572937099494,1: }
J ={}

{0: 0.2925924201509231,
 1: 0.24339477427955564,
 2: 0.032904766164551874,
 3: -0.13036292057109133}